In [1]:
import os
from glob import glob
import numpy as np
import numpy.ma as ma
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import lsst.sims.maf.metricBundles as mb
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers

In [4]:
import sys
sys.path.append(os.path.join(os.getenv('SIMS_MAF_DIR'), 'bin.src'))
import movingObjects as moO

In [44]:
runname = 'minion_1016'
ddir = os.path.join('neo_split_res', runname)
outdir = runname + 'neo'
orbitFile = 'orbits/neo_5k'

In [45]:
slicer = slicers.MoObjSlicer()
slicer.readOrbits(orbitFile, Hrange=np.arange(11, 28.5, 0.5))

In [46]:
pha_splits = os.listdir('orbits/neo_split')
pha_splits

['neo_5k_00',
 'neo_5k_01',
 'neo_5k_02',
 'neo_5k_03',
 'neo_5k_04',
 'neo_5k_05',
 'neo_5k_06',
 'neo_5k_07',
 'neo_5k_08',
 'neo_5k_09',
 'neo_5k_10']

In [50]:
# Read split results.
# ex: astro_lsst_01_1016_Discovery_Time_pha_5k_06_3_pairs_in_12_nights_MOOB.h5
strategy = ['3 pairs in 12 nights', '3 pairs in 15 nights', '3 pairs in 20 nights', 
            '3 pairs in 25 nights', '3 pairs in 30 nights', '4 pairs in 20 nights']
bundles = {}
for strat in strategy:
    bundles[strat] = {}
    for spl in pha_splits:
        filematch = os.path.join(ddir, runname + "_Discovery_Time*" + "*" + spl + "*" + 
                                 strat.replace(' ', '_') + "*h5")
        #print(filematch)
        filename = glob(filematch)[0]
        bundles[strat][spl] = mb.createEmptyMoMetricBundle()
        bundles[strat][spl].read(filename)
        Hrange = bundles[strat][spl].slicer.Hrange
        bundles[strat][spl].slicer = slicer
        bundles[strat][spl].slicer.Hrange = Hrange

In [51]:
# Join results.
times = np.arange(59579, 65055.15+5, 10)
summaryMetrics = [metrics.MoCompletenessAtTimeMetric(times=times, Hval=22,cumulative=False, Hindex=0.33),
                  metrics.MoCompletenessAtTimeMetric(times=times, Hval=22, cumulative=True, Hindex=0.33)]


timebundles = {}
for strat in strategy:
    filematch = os.path.join(ddir, runname + "_Discovery_Time*" + "*neo_5k_00*" + strat.replace(' ', '_') + "*h5")
    filename = glob(filematch)[0]
    timebundles[strat] = mb.createEmptyMoMetricBundle()
    timebundles[strat].read(filename)
    timebundles[strat].fileRoot = timebundles[strat].fileRoot.replace('_00', '')
    Hrange1 = timebundles[strat].slicer.Hrange
    timebundles[strat].slicer = slicer
    timebundles[strat].slicer.Hrange = Hrange1
    mvals = np.zeros((len(slicer.orbits), len(Hrange1)), float)
    mvalsMask = np.ones((len(slicer.orbits), len(Hrange1)), dtype=bool)
    for spl in pha_splits:
        if np.any(np.not_equal(bundles[strat][spl].slicer.Hrange, Hrange1)):
            raise ValueError('Incompatible results, Hrange varies')
        mvals = mvals + bundles[strat][spl].metricValues.filled(0)
        mvalsMask = np.where(mvalsMask & bundles[strat][spl].metricValues.mask, True, False)
    timebundles[strat].metricValues = ma.MaskedArray(data=mvals, mask=mvalsMask, fill_value=0)
    timebundles[strat].metadata = 'NEO ' + strat
    timebundles[strat].runname = runname
    timebundles[strat].setSummaryMetrics(summaryMetrics)
    timebundles[strat].computeSummaryStats()
    timebundles[strat].write()

In [52]:
diff = np.abs(times-63777.50)
idx = np.where(diff == np.min(diff))[0]
idx, times[idx]

(array([420]), array([ 63779.]))

In [27]:
# astro-lsst-01-1016
for strat in strategy:
    print(strat, timebundles[strat].summaryValues['CumulativeCompleteness@Time']['name'][idx],
         timebundles[strat].summaryValues['CumulativeCompleteness@Time']['value'][idx])

3 pairs in 12 nights ['H <=22.0 @ 63779.00'] [ 0.68204649]
3 pairs in 15 nights ['H <=22.0 @ 63779.00'] [ 0.69072919]
3 pairs in 20 nights ['H <=22.0 @ 63779.00'] [ 0.69913533]
3 pairs in 25 nights ['H <=22.0 @ 63779.00'] [ 0.706219]
3 pairs in 30 nights ['H <=22.0 @ 63779.00'] [ 0.71353221]
4 pairs in 20 nights ['H <=22.0 @ 63779.00'] [ 0.64137358]


In [28]:
# astro-lsst-01_1016 @ time 10 years
diff = np.abs(times-63230.5)
idx = np.where(diff == np.min(diff))[0]
idx, times[idx]
for strat in strategy:
    print(strat, timebundles[strat].summaryValues['CumulativeCompleteness@Time']['name'][idx],
         timebundles[strat].summaryValues['CumulativeCompleteness@Time']['value'][idx])

3 pairs in 12 nights ['H <=22.0 @ 63229.00'] [ 0.64639581]
3 pairs in 15 nights ['H <=22.0 @ 63229.00'] [ 0.65581696]
3 pairs in 20 nights ['H <=22.0 @ 63229.00'] [ 0.66542175]
3 pairs in 25 nights ['H <=22.0 @ 63229.00'] [ 0.6728706]
3 pairs in 30 nights ['H <=22.0 @ 63229.00'] [ 0.67995452]
4 pairs in 20 nights ['H <=22.0 @ 63229.00'] [ 0.6048947]


In [29]:
# astro-lsst-01_1016 @ time 15 years
diff = np.abs(times-65055.5)
idx = np.where(diff == np.min(diff))[0]
idx, times[idx]
for strat in strategy:
    print(strat, timebundles[strat].summaryValues['CumulativeCompleteness@Time']['name'][idx],
         timebundles[strat].summaryValues['CumulativeCompleteness@Time']['value'][idx])

3 pairs in 12 nights ['H <=22.0 @ 65059.00'] [ 0.72742593]
3 pairs in 15 nights ['H <=22.0 @ 65059.00'] [ 0.73537991]
3 pairs in 20 nights ['H <=22.0 @ 65059.00'] [ 0.74300583]
3 pairs in 25 nights ['H <=22.0 @ 65059.00'] [ 0.74990049]
3 pairs in 30 nights ['H <=22.0 @ 65059.00'] [ 0.75586918]
4 pairs in 20 nights ['H <=22.0 @ 65059.00'] [ 0.69090125]


In [36]:
# astro-lsst-01-1015
for strat in strategy:
    print(strat, timebundles[strat].summaryValues['CumulativeCompleteness@Time']['name'][idx],
         timebundles[strat].summaryValues['CumulativeCompleteness@Time']['value'][idx])

3 pairs in 12 nights ['H <=22.0 @ 63779.00'] [ 0.66120651]
3 pairs in 15 nights ['H <=22.0 @ 63779.00'] [ 0.67287054]
3 pairs in 20 nights ['H <=22.0 @ 63779.00'] [ 0.68373311]
3 pairs in 25 nights ['H <=22.0 @ 63779.00'] [ 0.69632779]
3 pairs in 30 nights ['H <=22.0 @ 63779.00'] [ 0.70674202]
4 pairs in 20 nights ['H <=22.0 @ 63779.00'] [ 0.60695386]


In [43]:
# astro-lsst-01-1017
for strat in strategy:
    print(strat, timebundles[strat].summaryValues['CumulativeCompleteness@Time']['name'][idx],
         timebundles[strat].summaryValues['CumulativeCompleteness@Time']['value'][idx])

3 pairs in 12 nights ['H <=22.0 @ 63779.00'] [ 0.69069903]
3 pairs in 15 nights ['H <=22.0 @ 63779.00'] [ 0.70007617]
3 pairs in 20 nights ['H <=22.0 @ 63779.00'] [ 0.71026322]
3 pairs in 25 nights ['H <=22.0 @ 63779.00'] [ 0.71924839]
3 pairs in 30 nights ['H <=22.0 @ 63779.00'] [ 0.72649177]
4 pairs in 20 nights ['H <=22.0 @ 63779.00'] [ 0.65098262]


In [53]:
# minion_1016
for strat in strategy:
    print(strat, timebundles[strat].summaryValues['CumulativeCompleteness@Time']['name'][idx],
         timebundles[strat].summaryValues['CumulativeCompleteness@Time']['value'][idx])

3 pairs in 12 nights ['H <=22.0 @ 63779.00'] [ 0.64600026]
3 pairs in 15 nights ['H <=22.0 @ 63779.00'] [ 0.65447927]
3 pairs in 20 nights ['H <=22.0 @ 63779.00'] [ 0.6626879]
3 pairs in 25 nights ['H <=22.0 @ 63779.00'] [ 0.66903789]
3 pairs in 30 nights ['H <=22.0 @ 63779.00'] [ 0.67542836]
4 pairs in 20 nights ['H <=22.0 @ 63779.00'] [ 0.61016482]
